In [ ]:
# Configs
%load_ext autoreload
%autoreload 3
import sys
sys.path.insert(0, "../")
from src_code.data_utils.dataset import GridDataset
from src_code.data_utils.dataset_utils import CellType
from src_code.data_utils.prompt_utils import prompt_generator
from src_code.eval_utils.eval import calculate_score, eval_results
from openai import OpenAI
from io import BytesIO
import base64
import ast

api_key = ""

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
from openai import OpenAI

client = OpenAI(
  api_key=api_key
)

completion = client.chat.completions.create(
  model="gpt-4o-mini",
  store=True,
  messages=[
    {"role": "user", "content": "write a haiku about ai"}
  ]
)

print(completion.choices[0].message)


ChatCompletionMessage(content='Beneath the circuits,  \nQuiet thoughts without a voice,  \nDreams of silicon.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)


In [39]:
def encode_pil_image(pil_img):
    """Converts PIL.Image.Image to base64 string (PNG format)"""
    buffered = BytesIO()
    pil_img.save(buffered, format="PNG")
    return base64.b64encode(buffered.getvalue()).decode("utf-8")

def inference(prompt, img=None):
    messages = [{"role": "user", "content": []}]

    if img is not None:
        base64_image = encode_pil_image(img)
        messages[0]["content"].append({"type": "text", "text": prompt})
        messages[0]["content"].append({
            "type": "image_url",
            "image_url": {
                "url": f"data:image/png;base64,{base64_image}"
            }
        })
    else:
        messages[0]["content"] = prompt  # just plain text

    completion = client.chat.completions.create(
        model="gpt-4o-mini",  # or gpt-4o-mini
        messages=messages if isinstance(messages[0]["content"], list) else [{"role": "user", "content": messages[0]["content"]}],
        temperature=0,
        timeout=20,
    )

    return completion.choices[0].message


In [40]:
dataset = GridDataset(grid_size=5, seed = 42, wall_symbol="#", free_symbol=".")

In [42]:
path_results_img = []
path_results_ascii = []
path_results_coor = []
path_results_img_raw = []
path_results_ascii_raw = []
path_results_coor_raw = []
EVAL_NUM = 100
for i in range(EVAL_NUM):
    img_rgb, grid_world = dataset[i]
    prompt_img = prompt_generator(grid_world, pure_language=False, img=True, img_symbol="This image")
    prompt_ascii = prompt_generator(grid_world, pure_language=False)
    prompt_coor = prompt_generator(grid_world, pure_language=True)
    path_result_img = inference(
        prompt_img, 
        img=img_rgb
    )
    patj_result_ascii = inference(
        prompt_ascii
    )
    path_result_coor = inference(
        prompt_coor
    )
    path_results_img_raw.append(path_result_img.content)
    path_results_ascii_raw.append(patj_result_ascii.content)
    path_results_coor_raw.append(path_result_coor.content)
    try:
        print(f"{path_result_img.content=}")
        path_result_img = ast.literal_eval(path_result_img.content)
        if type(path_result_img) == str:
            path_result_img = (path_result_img,)
        print(f"{path_result_img=}")
        path_results_img.append(path_result_img)
    except Exception as e:
        print(f"{e=}")
        print(f"Fail to parse path_result_img")
    try:
        print(f"{patj_result_ascii.content=}")
        path_result_ascii = ast.literal_eval(patj_result_ascii.content)
        if type(path_result_ascii) == str:
            path_result_ascii = (path_result_ascii,)
        path_results_ascii.append(path_result_ascii)
        print(f"{path_result_ascii=}")
    except Exception as e:
        print(f"{e=}")
        print(f"Fail to parse path_result_ascii")
    try:
        print(f"{path_result_coor.content=}")
        path_result_coor = ast.literal_eval(path_result_coor.content)
        if type(path_result_coor) == str:
            path_result_coor = (path_result_coor,)
        print(f"{path_result_coor=}")
        path_results_coor.append(path_result_coor)
    except Exception as e:
        print(f"{e=}")
        print(f"Fail to parse path_result_coor")
    print(f"{grid_world.a_star()=}\n")

path_result_img.content="('go up', 'go up')"
path_result_img=('go up', 'go up')
patj_result_ascii.content="('go down', 'go down', 'go right', 'go right', 'go up', 'go up')"
path_result_ascii=('go down', 'go down', 'go right', 'go right', 'go up', 'go up')
path_result_coor.content="('go up', 'go up', 'go left', 'go left', 'go down', 'go down', 'go left')"
path_result_coor=('go up', 'go up', 'go left', 'go left', 'go down', 'go down', 'go left')
grid_world.a_star()=('go up', 'go up', 'go up', 'go left')

path_result_img.content="('go up', 'go left', 'go left', 'go down', 'go down', 'go right', 'go right')"
path_result_img=('go up', 'go left', 'go left', 'go down', 'go down', 'go right', 'go right')
patj_result_ascii.content="('go down', 'go down', 'go down', 'go right', 'go right', 'go up', 'go up')"
path_result_ascii=('go down', 'go down', 'go down', 'go right', 'go right', 'go up', 'go up')
path_result_coor.content="('go up', 'go up', 'go left', 'go left')"
path_result_coor=('go up', '

In [43]:
eval_result_img = eval_results(path_results_img, dataset)
eval_result_ascii = eval_results(path_results_ascii, dataset)
eval_result_coor = eval_results(path_results_coor, dataset)
print(f"{eval_result_img=}")
print(f"{eval_result_ascii=}")
print(f"{eval_result_coor=}")
result = {"img": eval_result_img, "ascii": eval_result_ascii, "coor": eval_result_coor}

eval_result_img={'success rate (%)': 7.000000000000001, 'average collision': 0.79, 'average goal_distance': 3.0, 'average path_length_difference': 1.82}
eval_result_ascii={'success rate (%)': 0.0, 'average collision': 1.07, 'average goal_distance': 3.43, 'average path_length_difference': 2.93}
eval_result_coor={'success rate (%)': 48.0, 'average collision': 0.61, 'average goal_distance': 0.94, 'average path_length_difference': 1.1}


In [44]:
result

{'img': {'success rate (%)': 7.000000000000001,
  'average collision': 0.79,
  'average goal_distance': 3.0,
  'average path_length_difference': 1.82},
 'ascii': {'success rate (%)': 0.0,
  'average collision': 1.07,
  'average goal_distance': 3.43,
  'average path_length_difference': 2.93},
 'coor': {'success rate (%)': 48.0,
  'average collision': 0.61,
  'average goal_distance': 0.94,
  'average path_length_difference': 1.1}}

In [45]:
import json

with open('../result_gpt4omini_100.json', 'w') as f:
    json.dump(result, f, indent=4)

with open('../gpt4o_mini_results_img_raw.json', 'w') as f:
    json.dump(path_results_img_raw, f, indent=4)

with open('../gpt4o_mini_results_ascii_raw.json', 'w') as f:
    json.dump(path_results_ascii_raw, f, indent=4)

with open('../gpt4o_mini_results_coor_raw.json', 'w') as f:
    json.dump(path_results_coor_raw, f, indent=4)

In [5]:
%load_ext autoreload
%autoreload 3
import sys
sys.path.insert(0, "../")
import json
from src_code.data_utils.dataset import GridDataset
from src_code.eval_utils.eval import calculate_score, eval_results
import ast
dataset = GridDataset(grid_size=5, seed = 42, wall_symbol="#", free_symbol=".")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:

# read the results
path_results_img = []
path_results_ascii = []
path_results_coor = []
with open('../gpt4o_mini_results_img_raw.json', 'r') as f:
    path_results_img_raw = json.load(f)

with open('../gpt4o_mini_results_ascii_raw.json', 'r') as f:
    path_results_ascii_raw = json.load(f)

with open('../gpt4o_mini_results_coor_raw.json', 'r') as f:
    path_results_coor_raw = json.load(f)

for i, content in enumerate(path_results_img_raw):
    r = ast.literal_eval(content)
    if type(r) == str:
        r = (r,)
    path_results_img.append(r)

for i, content in enumerate(path_results_ascii_raw):
    r = ast.literal_eval(content)
    if type(r) == str:
        r = (r,)
    path_results_ascii.append(r)

for i, content in enumerate(path_results_coor_raw):
    r = ast.literal_eval(content)
    if type(r) == str:
        r = (r,)
    path_results_coor.append(r)

eval_result_img = eval_results(path_results_img, dataset)
eval_result_ascii = eval_results(path_results_ascii, dataset)
eval_result_coor = eval_results(path_results_coor, dataset)
print(f"{eval_result_img=}")
print(f"{eval_result_ascii=}")
print(f"{eval_result_coor=}")
result = {"img": eval_result_img, "ascii": eval_result_ascii, "coor": eval_result_coor}

# print(result)
# with open('../result_gpt4omini_100.json', 'w') as f:
#     json.dump(result, f, indent=4)

eval_result_img={'success rate (%)': 7.000000000000001, 'average collision': 0.79, 'average goal_distance': 3.0, 'average path_length_difference': 1.82, 'pass_through_goal': 19}
eval_result_ascii={'success rate (%)': 0.0, 'average collision': 1.07, 'average goal_distance': 3.43, 'average path_length_difference': 2.93, 'pass_through_goal': 22}
eval_result_coor={'success rate (%)': 48.0, 'average collision': 0.61, 'average goal_distance': 0.94, 'average path_length_difference': 1.1, 'pass_through_goal': 11}
